fake news detector

We are using A Million News Headlines dataset to implement a fake news headlines detection model. Where the A Million News Headlines dataset will be labeled as real news headlines. we will also use two fake news headline datasets on Kaggle from Fake and real news and Getting Real about Fake News .

In [3]:
# imports for reading our dataset
import numpy as np
import pandas as pd

load the dataset

In [4]:
#Reading the three Dataset
Headlines = pd.read_csv('/content/drive/MyDrive/fakenewsdata/abcnews-date-text.csv', usecols =["headline_text"]).dropna()
Headlines1 = pd.read_csv('/content/drive/MyDrive/fakenewsdata/Fake.csv', usecols =["title"]).dropna()
Headlines2 = pd.read_csv('/content/drive/MyDrive/fakenewsdata/True.csv', usecols =["title"]).dropna()

In [5]:
#Removing duplicated headlines from the data
Headlines = Headlines.drop_duplicates('headline_text')
Headlines1 = Headlines1.drop_duplicates('title')
Headlines2 = Headlines2.drop_duplicates('title')

In [6]:
#Renaming the dataframe columns and Combining all datasets
Headlines1 = Headlines1.rename(columns={'title': 'headline_text'})
Headlines2 = Headlines2.rename(columns={'title': 'headline_text'})

data labelling

In [7]:
#Creating lable for datasets
#million-headlines dataset will be used as real headlines
#fake-and-real-news-dataset & fake-news dataset will be used as fake headlines
Headlines['fake'] = 0
Headlines1['fake'] = 1
Headlines2['fake'] = 1

In [8]:
#combining dataset
#Downsize million-headlines dataset to first 50K rows
data = pd.concat([Headlines[:50000],Headlines1,Headlines2])
print('Training dataset contains: {} Real headlines and {} Fake headlines.'.format(50000,len(Headlines1)+len(Headlines2)))

Training dataset contains: 50000 Real headlines and 38729 Fake headlines.


so the dataset contains 50000 real headlines and 38729 fake headlines

data preprocessing

In [23]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
# importing important liabraries
import gensim
import nltk as nl
from sklearn.feature_extraction import _stop_words


nltk_stopwords = nl.corpus.stopwords.words('english')
gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS
sklearn_stopwords = _stop_words.ENGLISH_STOP_WORDS
combined_stopwords = sklearn_stopwords.union(nltk_stopwords,gensim_stopwords)

In [25]:
print('NLTK has {} stop words'.format(len(nltk_stopwords)))
print('Gensim has {} stop words'.format(len(gensim_stopwords)))
print('Sklearn has {} stop words'.format(len(sklearn_stopwords)))
print('Combined stopwords list has {} stop words'.format(len(combined_stopwords)))

NLTK has 179 stop words
Gensim has 337 stop words
Sklearn has 318 stop words
Combined stopwords list has 390 stop words


In [26]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()

In [27]:
data['headline_text'] = data['headline_text'].apply(lambda x: x.lower())
data['headline_text'] = data['headline_text'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))
data['headline_text'] = data['headline_text'].apply(lambda x: ' '.join([porter_stemmer.stem(word) for word in x.split()]))
data['headline_text'] = data['headline_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (combined_stopwords)]))

splitting the data
training data is 80 percent
testing data is 20 percent

In [28]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(data['headline_text'], data['fake'], test_size=0.2, random_state=7)

Construct models with TF-IDF

we are going to use the following models and test their accuracy


DecisionTreeClassifier,
LogisticRegression,
SVC,
KNeighborsClassifier and Naive_Bayes

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, MaxPooling1D, Flatten, Embedding, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [31]:
tfidf_vectorizer = TfidfVectorizer(tokenizer = word_tokenize, max_features = 300)
tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)
tfidf_features = tfidf_vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
svc = SVC(kernel='linear')
knn = KNeighborsClassifier()
nb = MultinomialNB()
lg=  LogisticRegression()

dt.fit(tfidf_train, y_train)
rf.fit(tfidf_train, y_train)
svc.fit(tfidf_train, y_train)
knn.fit(tfidf_train, y_train)
nb.fit(tfidf_train, y_train)
lg.fit(tfidf_train, y_train)

In [33]:
print ("Testing Acc. of Decision Tree: {} %".format(round(dt.score(tfidf_test, y_test) * 100, 2)))
print ("Testing Acc. of Random Forest: {} %".format(round(rf.score(tfidf_test, y_test) * 100, 2)))
print ("Testing Acc. of SVC: {} %".format(round(svc.score(tfidf_test, y_test) * 100, 2)))
print ("Testing Acc. of K-NN: {} %".format(round(knn.score(tfidf_test, y_test) * 100, 2)))
print ("Testing Acc. of Naive Bayesian: {} %".format(round(nb.score(tfidf_test, y_test) * 100, 2)))
print ("Testing Acc. of Logistic Regression: {} %".format(round(lg.score(tfidf_test, y_test) * 100, 2)))

Testing Acc. of Decision Tree: 83.75 %
Testing Acc. of Random Forest: 85.33 %
Testing Acc. of SVC: 84.55 %
Testing Acc. of K-NN: 82.25 %
Testing Acc. of Naive Bayesian: 83.79 %
Testing Acc. of Logistic Regression: 85.11 %
